In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

# img = load_img('./data/train/cats/cat.0.jpg')  # this is a PIL image
# x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
# x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# # the .flow() command below generates batches of randomly transformed images
# # and saves the results to the `preview/` directory
# i = 0
# for batch in datagen.flow(x, batch_size=1,
#                           save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
#     i += 1
#     if i > 20:
#         break  # otherwise the generator would loop indefinitely

Using TensorFlow backend.


In [2]:

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 300
epochs = 10
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [3]:

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 24000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [4]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
125/125 [==============================] - 63s 502ms/step - loss: 0.6946 - acc: 0.4965 - val_loss: 0.6913 - val_acc: 0.5104
Epoch 2/10
125/125 [==============================] - 60s 477ms/step - loss: 0.6927 - acc: 0.5165 - val_loss: 0.6885 - val_acc: 0.5451
Epoch 3/10
125/125 [==============================] - 59s 475ms/step - loss: 0.6915 - acc: 0.5200 - val_loss: 0.6869 - val_acc: 0.6285
Epoch 4/10
125/125 [==============================] - 61s 490ms/step - loss: 0.6890 - acc: 0.5280 - val_loss: 0.6824 - val_acc: 0.5500
Epoch 5/10
125/125 [==============================] - 62s 497ms/step - loss: 0.6859 - acc: 0.5610 - val_loss: 0.6818 - val_acc: 0.5868
Epoch 6/10
125/125 [==============================] - 59s 470ms/step - loss: 0.6842 - acc: 0.5635 - val_loss: 0.6824 - val_acc: 0.5139
Epoch 7/10
125/125 [==============================] - 59s 472ms/step - loss: 0.6800 - acc: 0.5645 - val_loss: 0.6698 - val_acc: 0.6214
Epoch 8/10
125/125 [==============================] - 5

In [5]:
model.summary()
model.save_weights('model_wieghts2.h5')
model.save('model_keras2.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18496)             0         
__________

In [6]:
test_data_dir = 'data/test'

test_generator = test_datagen.flow_from_directory(
    test_data_dir, 
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,  # only data, no labels
    shuffle=False)

filenames = test_generator.filenames
nb_samples = len(filenames)
print(nb_samples)

Found 12500 images belonging to 1 classes.
12500


In [7]:
prediction_probabilities = model.predict_generator(test_generator, 2000)

In [ ]:
import pandas as pd

counter = range(1, len(prediction_probabilities) + 1)
solution = pd.DataFrame({"id": counter, "label":list(prediction_probabilities)})
cols = ['label']

for col in cols:
    solution[col] = solution[col].map(lambda x: str(x).lstrip('[').rstrip(']')).astype(float)

solution.to_csv("dogsVScats.csv", index = False)

In [8]:
i = 0
for batch in test_datagen.flow_from_directory(
    test_data_dir, save_to_dir='images', save_prefix='hi',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary', shuffle=False):
    i += 1
    if i > 20: # save 20 images
        break  # otherwise the generator would loop indefinitely


Found 12500 images belonging to 1 classes.


In [10]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary', shuffle=False)

loss, acc = model.evaluate_generator(test_generator, nb_train_samples // batch_size)

print('Test loss:', loss)
print('Test accuracy:', acc)

Found 12500 images belonging to 1 classes.
Test loss: 0.7550857090950012
Test accuracy: 0.4
